<a href="https://colab.research.google.com/github/chrisnjw/Chris_Portfolio_DS/blob/master/Credit_card_fraud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
# Loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [3]:
# Printing the first five row of the data set
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [4]:
# Printing the last five row of the data set
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
114957,73690,-2.197480,1.982629,0.259502,0.924323,-0.879938,-0.135952,-0.380186,0.611134,0.278764,0.746228,-1.427950,-0.025305,-0.206072,0.254171,0.612202,-0.723040,0.762576,-0.117120,2.178454,0.045666,-0.033516,-0.367199,-0.099698,-0.111166,-0.182825,0.427261,-0.895134,0.164611,9.51,0.0
114958,73690,-2.197480,1.982629,0.259502,0.924323,-0.879938,-0.135952,-0.380186,0.611134,0.278764,0.746228,-1.427950,-0.025305,-0.206072,0.254171,0.612202,-0.723040,0.762576,-0.117120,2.178454,0.045666,-0.033516,-0.367199,-0.099698,-0.111166,-0.182825,0.427261,-0.895134,0.164611,9.51,0.0
114959,73690,1.255655,0.293362,0.288616,0.701727,-0.447134,-1.093442,0.029565,-0.155947,0.160172,-0.257903,-0.209758,-0.213470,-0.617437,-0.119662,1.186461,0.518032,0.042988,-0.195785,-0.192785,-0.134786,-0.297287,-0.898954,0.139494,0.322281,0.187536,0.097228,-0.028586,0.029014,1.79,0.0
114960,73690,1.270638,-0.089535,-0.990690,-0.375303,0.652307,-0.073908,0.394543,-0.164288,-0.317251,0.054441,-0.062050,0.352671,0.383281,0.559560,0.178596,0.610500,-1.043495,0.364609,1.037631,0.136945,-0.067492,-0.344301,-0.372630,-1.296908,0.731021,1.152123,-0.131651,-0.024591,75.00,0.0
114961,73691,1.295784,0.049457,-1.301814,-0.476648,2.131357,3.184446,-0.494416,0.802781,-0.075014,-0.221156,0.131691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Dataset informations
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114962 entries, 0 to 114961
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    114962 non-null  int64  
 1   V1      114962 non-null  float64
 2   V2      114962 non-null  float64
 3   V3      114962 non-null  float64
 4   V4      114962 non-null  float64
 5   V5      114962 non-null  float64
 6   V6      114962 non-null  float64
 7   V7      114962 non-null  float64
 8   V8      114962 non-null  float64
 9   V9      114962 non-null  float64
 10  V10     114962 non-null  float64
 11  V11     114962 non-null  float64
 12  V12     114961 non-null  float64
 13  V13     114961 non-null  float64
 14  V14     114961 non-null  float64
 15  V15     114961 non-null  float64
 16  V16     114961 non-null  float64
 17  V17     114961 non-null  float64
 18  V18     114961 non-null  float64
 19  V19     114961 non-null  float64
 20  V20     114961 non-null  float64
 21  V21     11

In [6]:
# Checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [7]:
# Replacing the null value with empty string
credit_card_data = credit_card_data.fillna('')

In [8]:
# Double checking for whether there is any missing value
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [9]:
# Distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    114719
1.0       242
            1
Name: Class, dtype: int64

This dataset is highly unbalanced

0 --> Normal Transaction

1 --> fraudulent transaction

In [10]:
# Separating the data for analysis
legit = credit_card_data[credit_card_data.Class == 0.0]
fraud = credit_card_data[credit_card_data.Class == 1.0]

In [11]:
print(legit.shape)
print(fraud.shape)

(114719, 31)
(242, 31)


In [12]:
# Statistical measures of the data
legit.Amount.describe()

count     114719.0
unique     21006.0
top            1.0
freq        4732.0
Name: Amount, dtype: float64

In [14]:
# Statustucal measures of the data
fraud.Amount.describe()

count     242.0
unique    118.0
top         1.0
freq       72.0
Name: Amount, dtype: float64

In [15]:
# compare the values for both transaction
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
Class,,,,,,,,,,,,
0.0,46095.931415,-0.242322,-0.023224,0.698223,0.145973,-0.274923,0.096444,-0.102075,0.055820,-0.063634,-0.005629,0.220439
1.0,39270.280992,-5.942308,4.133122,-7.591109,4.716339,-4.227089,-1.524018,-6.307479,1.589037,-2.723048,-5.928648,4.288542
,73691.000000,1.295784,0.049457,-1.301814,-0.476648,2.131357,3.184446,-0.494416,0.802781,-0.075014,-0.221156,0.131691


Under-Sampling

Build a sample dataset containing similar distribution of normal transactions 
and Fraudulent Transactions

Number of Fraudulent Transactions --> 242

In [17]:
legit_sample = legit.sample(n = 242)

Concatenating two DataFrames

In [18]:
new_dataset = pd.concat([legit_sample, fraud], axis=0)

In [19]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
75926,56311,1.244712,-1.394011,-0.020311,-1.393230,0.452561,4.223746,-2.006395,1.155012,0.248825,0.445422,-0.826146,-0.000967229,0.391659,-0.739976,0.326046,-0.893595,-0.394139,1.8682,-0.419372,-0.245403,-0.255372,-0.256581,-0.057289,1.0632,0.249981,0.989364,0.0287358,0.0314749,60,0
24332,33179,-1.750985,1.321066,-1.085649,1.695364,-3.544851,1.717507,2.991449,-0.219218,-0.341416,0.418283,-0.283133,-0.0360662,0.416931,0.406467,1.90356,-0.399519,0.368798,-0.183592,0.867288,-1.12131,-0.0492596,1.12836,0.166649,0.186287,-1.12073,-0.27114,0.0266927,-0.335486,655.98,0
75666,56189,-1.374949,0.364229,2.158245,-0.513601,1.645626,-0.442264,1.424071,-0.626983,0.159339,-0.293276,0.100819,0.108403,-0.491665,-0.585565,-0.515971,-0.829718,-0.334227,-1.59057,-2.37668,-0.30055,-0.0762263,0.386109,-0.600015,0.108488,0.679655,-0.586214,-0.670881,-0.597252,9.67,0
110101,71678,-1.417264,1.550366,0.624611,-0.422297,-0.189646,-1.123044,0.419491,0.676537,-1.379184,-0.896609,1.227594,1.38815,0.502217,1.03852,-0.708482,0.514886,-0.381221,-0.330897,-0.207923,-0.316094,-0.12308,-0.800219,0.0610018,0.520985,-0.226382,-0.191152,-0.340388,-0.049865,10.2,0
114207,73380,0.366938,-0.684030,0.336175,2.903367,-0.720543,-0.620429,0.753048,-0.259207,-0.843758,0.485093,-0.214403,0.0652779,0.0327382,0.42755,0.480135,0.632504,-0.470556,-0.490946,-1.16623,0.678406,0.0787098,-0.7768,-0.237028,0.687584,0.156017,-0.251518,-0.0901112,0.100511,418.35,0


In [20]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
108258,70828,0.196707,1.189757,0.704882,2.891388,0.045555,1.245730,-1.198714,-2.421616,-1.232089,0.324239,-1.273935,-0.868686,-1.18194,1.02758,1.68813,0.256216,0.120608,-0.052346,0.329827,0.646616,-1.32813,0.189311,-0.00552418,-0.814708,0.400924,0.286281,0.135215,0.257315,0.76,1
108708,71033,-3.170818,0.185735,-3.399852,3.761155,-2.148047,-1.598944,-2.519565,1.316215,-2.400106,-4.993417,4.447192,-5.29376,-1.41958,-6.42528,0.914083,-3.5152,-6.34745,-0.903512,1.19161,1.93225,1.09205,-0.0410804,0.904395,0.180016,0.0499703,-0.2571,0.859282,0.225924,426.4,1
111690,72327,-4.198735,0.194121,-3.917586,3.920748,-1.875486,-2.118933,-3.614445,1.687884,-2.189871,-4.684233,4.376907,-5.00744,-1.30475,-6.19248,1.0024,-3.31693,-6.18883,-1.04041,1.23304,1.00335,0.801312,-0.183001,-0.440387,0.292539,-0.144967,-0.251744,1.24941,-0.131525,238.9,1
112840,72824,-1.111495,-0.257575,2.250210,1.152671,0.432904,1.254126,-0.584163,-0.609682,1.014602,0.334533,0.826375,0.196887,-1.88599,-0.472026,-0.578141,-1.24301,0.57046,-0.159057,0.407188,-0.510614,0.862913,0.927825,-0.343058,-0.256268,-0.600742,-0.180331,0.0267622,-0.358335,45.03,1
114271,73408,-2.869795,1.335667,-1.009530,1.693885,-0.741480,-0.796773,-2.614242,1.066636,-1.135497,-3.943337,2.572830,-2.28866,0.562084,-5.33871,-1.85325,-2.61923,-5.23143,-1.23485,0.397308,0.0115572,0.528421,0.228027,-0.749298,-0.067179,0.215792,-0.453651,0.349191,0.198862,16.48,1


In [21]:
new_dataset['Class'].value_counts()

1.0    242
0.0    242
Name: Class, dtype: int64

In [22]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
Class,,,,,,,,,,,,
0.0,45786.772727,-0.146175,-0.000282,0.649773,0.154011,-0.133138,0.082419,-0.054143,0.020623,-0.088850,-0.085983,0.270130
1.0,39270.280992,-5.942308,4.133122,-7.591109,4.716339,-4.227089,-1.524018,-6.307479,1.589037,-2.723048,-5.928648,4.288542


Splitting the data into Features and Targets

In [32]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']
Y=Y.astype('int')

In [33]:
print(X)

         Time        V1        V2  ...        V27        V28  Amount
75926   56311  1.244712 -1.394011  ...  0.0287358  0.0314749      60
24332   33179 -1.750985  1.321066  ...  0.0266927  -0.335486  655.98
75666   56189 -1.374949  0.364229  ...  -0.670881  -0.597252    9.67
110101  71678 -1.417264  1.550366  ...  -0.340388  -0.049865    10.2
114207  73380  0.366938 -0.684030  ... -0.0901112   0.100511  418.35
...       ...       ...       ...  ...        ...        ...     ...
108258  70828  0.196707  1.189757  ...   0.135215   0.257315    0.76
108708  71033 -3.170818  0.185735  ...   0.859282   0.225924   426.4
111690  72327 -4.198735  0.194121  ...    1.24941  -0.131525   238.9
112840  72824 -1.111495 -0.257575  ...  0.0267622  -0.358335   45.03
114271  73408 -2.869795  1.335667  ...   0.349191   0.198862   16.48

[484 rows x 30 columns]


In [34]:
print(Y)

75926     0
24332     0
75666     0
110101    0
114207    0
         ..
108258    1
108708    1
111690    1
112840    1
114271    1
Name: Class, Length: 484, dtype: int64


Split the data into Training data & Testing data

In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [36]:
print(X.shape, X_train.shape, X_test.shape)

(484, 30) (387, 30) (97, 30)


Model Training

Logistic Regression

In [37]:
model = LogisticRegression()

In [38]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

Model Evaluation

Accuracy Score

In [39]:
# Accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy on Training Data {}'.format(training_data_accuracy))

Accuracy on Training Data 0.9431524547803618


In [40]:
# Accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy score on Test Data : {}'.format(test_data_accuracy))

Accuracy score on Test Data : 0.9381443298969072
